<a href="https://colab.research.google.com/github/aaronmat1905/MLdiaries/blob/main/Artificial%20Neural%20Networks/ArtificialNeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

**Assume that:**
- **Polynomial Type**:
$$
y = 0.96x^2 + 7.35x + 7.07
$$
- **Noise Level**:
$$
ϵ \sim N(0, 2.42)
$$
- **Architecture**: \
Input(1) → Hidden(32) → Hidden(72) → Output(1)
- **Learning Rate (α)** = 0.005
- **Architecture**: Narrow-To-Wide Architecture

# **Data Loading**

In [2]:
# Loading in our dataset: <Synthetically Generated>
# Loading in our dataset: <Synthetically Generated>
datasetPath = "ann_dataset.csv"
ann_data = pd.read_csv(datasetPath)
ann_data.info()
print("\n+++++++++++++++++++++++++\n")
ann_data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   x       100000 non-null  float64
 1   y       100000 non-null  float64
dtypes: float64(2)
memory usage: 1.5 MB

+++++++++++++++++++++++++



,x,y
0,-55.601366,2547.648326
1,74.146461,5803.114349
2,-58.656169,2864.092846
3,83.722182,7324.122723
4,-2.317762,-1.245122


# **Activation Functions**
**ReLU**:

*Rectified Linear Unit*
$$
f(x) = max(0, x)
$$
- The ReLU function returns the maximum between between it's input and zero.
- Advantages:
  - It helps mitigate the Vanishing gradient problem.
  - Leads to efficient computation, since, It eliminates all negative outputs.
  - Allows NNs to scale to many layers without a significant increase in computational burden.

___
**ReLU Derivative**
The derivative of a function measures how much it changes it's slope:

*Here, we have to measure it's slope/derivative of the function in each region*

**Case Analysis:**
- Case 1: 𝒳 > 0:
  - In this region, f(𝒳) = 𝒳 ; **f'(𝒳) = 1**
- Case 2: 𝒳 < 0:
  - In this region, f(𝒳) = 0 ; **f'(𝒳) = 0**
- Case 3: 𝒳 = 0:
  - Left Hand Derivative: = 0
  - Right Hand Derivative: = 1
  - Therefore, the derivative at 𝒳 = 0 is undefined.


In [3]:
def relu(z):
  return max(0, z)

def relu_derivative(z):
  answer = relu(z)
  if answer < 0:
    return 0
  elif answer > 0:
    return 1
  else:
    return None

# **Loss Function**
**MSE**:

MSE, or Mean Squared error measures the average squared difference between the actual observed values and values predicted by a model.
- Smaller MSE ⇒ Models predictions closer to the actual data.

$$
\mathcal{L}_{\text{MSE}} = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2
$$

In [5]:
def MSE(y_true, y_pred):
  # Assuming y_true and y_pred are numpy arrays
  N = len(y_true)
  return (1/N)*np.sum(y_true - y_pred)

# **Weight Initialization**
> Weight initialization is the process of setting the starting values of the weights (parameters) in a neural network before the training begins.

Since training relies on *gradient descent*, the initial weights heavily influence the:
  - Convergence speed (How fast training progresses)
  - Avoiding poor local minima
  - Preventing vanishing or exploding gradients in deep network.

**Why not start with all 0s?**

If so, every neuron in a layer learns the same thing (*symmetry problem*). Gradient updates will be identical, so network won't learn effectively.



---

## **Xavier (Glorot) Initialization**

### ***The Core Idea***

A neuron’s output is a weighted sum of its inputs:

$$
z_j^{(l)} = \sum_{i=1}^{n_{in}} W_{ji}^{(l)} x_i^{(l-1)}
$$

Here:

* $n_{in}$ = number of inputs (fan-in)
* $n_{out}$ = number of outputs (fan-out)
* $W_{ji}$ = weights
* $x^{(l-1)}$ = activations from the previous layer

For stable training, we want two things:

1. The **variance of activations** remains consistent across layers (forward stability).
2. The **variance of gradients** remains consistent across layers (backward stability).

---

### ***Forward Stability***

The variance of the output $z_j^{(l)}$ depends on the variance of weights and inputs:

$$
Var[z_j^{(l)}] = n_{in} \cdot Var[W] \cdot Var[x^{(l-1)}]
$$

To avoid activations shrinking or exploding, we want:

$$
Var[z_j^{(l)}] \approx Var[x^{(l-1)}]
$$

This gives:

$$
Var[W] = \frac{1}{n_{in}}
$$

---

### ***Backward Stability***

During backpropagation, gradients flow through weights as well. Their variance depends on $n_{out}$:

$$
Var\left[\frac{\partial L}{\partial x^{(l-1)}}\right] = n_{out} \cdot Var[W] \cdot Var\left[\frac{\partial L}{\partial z^{(l)}}\right]
$$

For stable gradients:

$$
Var[W] = \frac{1}{n_{out}}
$$

---

### ***The Compromise***

* Forward pass prefers: $Var[W] = \frac{1}{n_{in}}$
* Backward pass prefers: $Var[W] = \frac{1}{n_{out}}$

Xavier/Glorot initialization takes a **compromise** between the two:

$$
Var[W] = \frac{2}{n_{in} + n_{out}}
$$

---

### ***Practical Formulations***

From this variance, we can define two initialization schemes:

* **Uniform Distribution**

$$
W \sim U\left(-\sqrt{\frac{6}{n_{in}+n_{out}}}, \; \sqrt{\frac{6}{n_{in}+n_{out}}}\right)
$$

* **Normal Distribution**

$$
W \sim \mathcal{N}\left(0, \frac{2}{n_{in}+n_{out}}\right)
$$

---

✅ **Summary:**
Xavier Initialization is a weight initialization method that ensures both forward activations and backward gradients maintain a stable variance throughout the network, preventing vanishing or exploding signals. It achieves this by balancing between the input and output layer sizes, setting the weight variance to:

$$
Var[W] = \frac{2}{n_{in} + n_{out}}
$$

Pseudocode for the Xavier-init method:
```
function Xavier_Init(n_in, n_out, distribution):
    if distribution == "uniform":
        limit = sqrt(6 / (n_in + n_out))
        W = random_uniform(-limit, +limit, size=(n_out, n_in))
    else if distribution == "normal":
        sigma = sqrt(2 / (n_in + n_out))
        W = random_normal(0, sigma, size=(n_out, n_in))
    return W
```

In [6]:
def xavier_initialization(data, input_dim, hidden1, hidden2, output_dim):
  seed = data[1][0]
  np.random.seed(seed)

  # Input -> Hidden1
  xavier_std1 = np.sqrt(2/(input_dim+hidden1))
  w1 = np.random.normal(0, xavier_std1, (hidden1, input_dim)) # Initialize weights
  b1 = np.zeroes((hidden1, 1)) # Initializing Biases

  # Hidden1 -> Hidden2
  xavier_std2 = np.sqrt(2/(hidden1+hidden2))
  w2 = np.random.normal(0, xavier_std2, (hidden2, hidden1))
  b2 = np.zeroes((hidden2, 1))

  # Hidden2 -> Output
  xavier_std3 = np.sqrt(2/(hidden2+output_dim))
  w3 = np.random.normal(0, xavier_std3, (output_dim, hidden2))
  b3 = np.zeroes((output_dim, 1))

  return w1, b1, w2, b2, w3, b3

# **Forward Propogation**
Forward propogation is the process by which **input data is passed through a neural network to compute output predictions**.

It involves performing linear transformations followed by non-linear activations at each layer.

Consider a feedforward neural network with:

* Input layer: $X \in \mathbb{R}^{n \times d_{\text{in}}}$
* Hidden layers: with weights $W^{[l]}$ and biases $b^{[l]}$
* Output layer: with weights $W^{[L]}$ and biases $b^{[L]}$

**The Steps are:**
1. Linear Transformation (Pre-activation)

For the $l$-th layer:

$$
z^{[l]} = a^{[l-1]} W^{[l]} + b^{[l]}
$$

* $a^{[l-1]}$ is the activation from the previous layer (or input $X$ for the first layer).
* $W^{[l]}$ are the layer weights, $b^{[l]}$ are biases.
* $z^{[l]}$ is called the **pre-activation** of layer $l$.

2. Activation Function:

After computing the pre-activation, $$a^{[l]} = g(z^{[l]})$$

The activation function introduces non-linear capabilities, allowing the network to model complex relationships

3. Output Layer:

For Regression Tasks, the output layer is often linear.
$$y^{^}=z[L]$$

For classification, a softmax or sigmoid activation is applied to produce probabilities.

Sure! Here’s a **standard theory explanation of forward propagation** in neural networks, framed formally like you’d see in a textbook or lecture notes:

___

### **4. Summary of Forward Pass**

1. Input $X$ → Linear transformation → Pre-activation $z^{[1]}$
2. Apply activation → $a^{[1]}$
3. Repeat for all hidden layers
4. Compute output layer → $\hat{y}$

**Vectorized form** ensures efficient computation across the entire batch.


### **Key Points**

* Forward propagation is **deterministic**: given inputs and weights, outputs are uniquely determined.
* It forms the **first step of training**: outputs are compared with true labels to compute the loss.
* The computed activations are also used in **backpropagation** to update weights.

In [7]:
def forwardPass(X, w1, b1, w2, b2, w3, b3):
  """
  Input -> Hidden1(Relu) -> Hidden2(Relu) --> Output(Linear)

  Returns:
    Pre-activations and activations for each layer
  """
  z1 = X @ w1 + b1
  a1 = relu(z1)

  z2 = a1 @ w2 + b2
  a2 = relu(z2)

  z3 = a2 @ w3 + b3

  return z1, a1, z2, a2, z3

# **Backward Propogation**